# UNIDADE 5:  Redes Neurais

<div style="border: 2px solid #00008B; padding: 15px; border-radius: 10px; background-color: #00008B; color: #FFFFFF; font-family: Arial;">
  <h1 style="margin-top: 0;"> Predição de Tempo de Atravessamento de Ações na Justiça</h1>
  <p>Descrição: Prever o tempo de atravessamento do processo (da solicitação até a efetiva entrega) utilizando informações históricas e considerando as características do processo no momento de autuação.
</p>
</div>

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
  <ul>
    <li>Rito (valores: trabalhista ou sumaríssimo)</li>
    <li>Tempo de Serviço do Reclamante (valores: tempo em meses até a data da despensa)</li>
    <li>Último salário do reclamante  (valores: número real)</li>
    <li>Profissão do reclamante (valores: comércio, indústria ou serviço)</li>
    <li>Cargo do reclamante (valores: direção ou execução)</li>
    <li>Objeto do processo (valores: falta de registro em carteira, diferença salarial, verbas recisórias, multa do Art, 477, multa do Art. 467, horas extras e reflexos, fundo de garantia por tempo de serviço, indenização por dados morais, seguro desemprego, vale transporte, adicional de insalubridade, adicional noturno, plano de saúde)</li>
    <li>Quantidade de depoimentos em cada audiência (valores: número inteiro entre 1 e 200)</li>
    <li>Acordo (valores: presença ou ausência)</li>
    <li>Necessidade de perícia (valores: S para Sim e N para Não)</li>
    <li>Solicitação de recurso ordinário contra sentença emitida pelo Juiz de 1 grau  (valores: S para Sim e N para Não)</li>
    <li>Solicitação de recurso de revista contra acordão (valores: S para Sim e N para Não)</li>
    <li>Número de audiências até a emissão da sentença (valores: número inteiro entre 1 e 200)</li>
    <li>Tempo médio de cada audiência (valor inteiro em minutos entre 30 e 1000)</li>
    <li>Duração do processo (valor inteiro em meses entre 1 e 500)</li>
 </ul>
</div>


In [ ]:
#!pip install torch torchvision torchaudio

In [ ]:
import torch
print(torch.__version__)

In [ ]:
import pandas as pd
import numpy as np

import streamlit as st
import joblib

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

### Carregando e preparando os dados

In [ ]:
df_normalizado = pd.read_csv("../data/processed/tempo_audiencias.csv")

In [ ]:
df_normalizado.columns

In [ ]:
X = df_normalizado.loc[:, df_normalizado.columns != 'duracao_processo']
y = df_normalizado["duracao_processo"]

In [ ]:
# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

### Convertendo para tensores do PyTorch

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
Um tensor é uma estrutura de dados multi-dimensional, semelhante a arrays e matrizes, mas com a vantagem de que pode ser utilizado em GPUs para acelerar cálculos intensivos. Tensors são a unidade básica para a manipulação de dados em PyTorch, por exemplo.
</div>

 *   **torch.tensor** converte o array para um tensor do PyTorch especificando que o tipo de dado dos elementos do tensor será float32.
 *   **view(-1, 1)** redimensiona o tensor para ter uma dimensão de (n, 1), onde n é o número de exemplos array. O -1 permite que o PyTorch infira automaticamente o tamanho dessa dimensão. O redimensionamento das labels com .view(-1, 1) é uma prática comum para garantir que as labels tenham a forma adequada para <span style="color:red">a maioria dos algoritmos de aprendizado supervisionado, que geralmente esperam que os labels sejam vetores colunares</span>.

### Definição da arquitetura da rede

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
O método forward define como os dados fluem através da rede. Ele recebe um tensor x e aplica as camadas da rede, bem como funções de ativação.
</div>

* **nn.Linear(n, m)** define uma camada totalmente conectada (fully connected) com n entradas e m saídas
* **torch.relu(self.fc1(x))** Aplica a primeira camada (self.fc1) aos dados de entrada x e, em seguida, aplica a função de ativação ReLU (Rectified Linear Unit).

In [ ]:
# Definindo o modelo da rede neural


In [ ]:
# Instanciando o modelo


### Função de Perda

A função de perda mede a diferença entre as previsões do modelo e os valores reais dos dados. Durante o treinamento, o objetivo é minimizar essa perda ajustando os pesos da rede neural.

#### MSELoss

`nn.MSELoss` é uma classe do PyTorch que implementa a função de perda de erro quadrático médio (Mean Squared Error, MSE). 

#### Fórmula do MSE

A fórmula do MSE para um conjunto de \( n \) exemplos é dada por:

$$ \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 $$



Durante o treinamento, a função de perda é usada da seguinte maneira:

*  **Forward Pass:** Os dados de entrada são passados através da rede para obter as previsões.
*  **Cálculo da Perda:**  A função de perda é usada para calcular a diferença entre as previsões e os valores reais.
*  **Backward Pass:**  O gradiente da perda em relação aos pesos da rede é calculado (backpropagation).
*  **Atualização dos Pesos:**  Os pesos da rede são atualizados para minimizar a perda.


In [ ]:
# Instanciando a função de perda 
criterion = nn.MSELoss()

### Otimizadores

Otimizadores são usados para atualizar os parâmetros do modelo (ou seja, os pesos da rede neural) com base no gradiente calculado durante o processo de backpropagation. O objetivo do otimizador é minimizar a função de perda ajustando os parâmetros do modelo.

`Adam Optimizer` é uma estratégia de otimização, Adam (abreviação de Adaptive Moment Estimation) combina as vantagens dos algoritmos AdaGrad e RMSProp, sendo eficiente em termos de memória e bem adequado para problemas com grandes dados e parâmetros.

*  **model.parameters()** retorna um iterador sobre os parâmetros do modelo que serão otimizados. Estes parâmetros são os pesos e vieses (biases) das várias camadas da rede neural definidas no modelo model.
*  **lr** significa "learning rate" (taxa de aprendizado). A taxa de aprendizado é um hiperparâmetro que controla o tamanho dos passos que o otimizador dá ao ajustar os parâmetros do modelo.

In [ ]:
# Instanciando o otimizador
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Treinando o modelo

1. **model.train()**
Coloca o modelo em modo de treinamento. Em PyTorch, certos módulos, como dropout e batch normalization, comportam-se de maneira diferente durante o treinamento e a inferência. O método model.train() garante que esses módulos estejam no modo de treinamento.

2. **outputs = model(X_train)**
Dados de entradas são passados pelo modelo para obter as previsões. Esta é a fase de "forward pass" onde o modelo gera suas saídas com base nos pesos atuais.

3. **loss = criterion(outputs, y_train)**
A função de perda é calculada comparando as previsões (outputs) com os valores reais (y_train). Esta perda quantifica o quão bem o modelo está performando.

4. **optimizer.zero_grad()**
Antes de realizar a backpropagation, é necessário zerar os gradientes acumulados dos passos anteriores. Isso é feito para evitar a acumulação de gradientes de múltiplas iterações.

5. **loss.backward()**
Realiza a backpropagation, calculando os gradientes da perda em relação aos parâmetros do modelo. Esses gradientes serão usados para atualizar os parâmetros.

6. **optimizer.step()**
Atualiza os parâmetros do modelo com base nos gradientes calculados durante a backpropagation. Este é o passo onde o otimizador ajusta os pesos para minimizar a função de perda.

In [ ]:
num_epochs = 500
for epoch in range(num_epochs):

    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

1. **model.eval()**
Coloca o modelo em modo de avaliação. Isto é importante porque certos módulos, como dropout e batch normalization, se comportam de maneira diferente durante o treinamento e a inferência. No modo de avaliação, dropout é desativado e batch normalization usa as estatísticas calculadas durante o treinamento.

2. **with torch.no_grad()**
Este contexto desabilita o cálculo de gradientes. Durante a avaliação, não é necessário calcular gradientes porque não estamos ajustando os pesos do modelo. Isso economiza memória e computação.

3. **model(X_test)**
Os dados de entrada de teste X_test são passados pelo modelo para obter as previsões. Esta é a fase de "forward pass" onde o modelo gera suas saídas com base nos pesos treinados.

4. **criterion(predictions, y_test)**
A função de perda é calculada comparando as previsões (predictions) com os valores reais de teste (y_test). Esta perda quantifica o quão bem o modelo está performando nos dados de teste.

### Avaliando o modelo

In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)
    print(f'Perda: {test_loss.item():.4f}')

### Exibindo as previsões

**Métrica de Performance:** O valor 0.0155 é o resultado da função de perda aplicada aos dados de teste. Ele quantifica a média dos quadrados das diferenças entre as previsões do modelo e os valores reais. Um valor mais baixo de perda indica que as previsões do modelo estão mais próximas dos valores reais.

In [ ]:
print(predictions[:10])

In [ ]:
torch.save(model, '../models/model.pth')

In [ ]:
#!streamlit run ../app.py